In [2]:
# %load ../../../standard_import.txt
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# load MATLAB files
from scipy.io import loadmat

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_seq_items', None)
 
#%config InlineBackend.figure_formats = {'pdf',}
%matplotlib inline

import seaborn as sns
sns.set_context('notebook')
sns.set_style('darkgrid')

#### Load MATLAB datafiles

In [3]:
data = loadmat('data/ex4data1.mat')
data.keys()

dict_keys(['__globals__', 'X', '__version__', 'y', '__header__'])

In [4]:
y = data['y']
# Add intercept
X = np.c_[np.ones((data['X'].shape[0],1)), data['X']]

print('X:',X.shape, '(with intercept)')
print('y:',y.shape)

X: (5000, 401) (with intercept)
y: (5000, 1)


In [5]:
weights = loadmat('data/ex3weights.mat')
weights.keys()

dict_keys(['__globals__', 'Theta2', 'Theta1', '__version__', '__header__'])

In [101]:
theta1, theta2 = weights['Theta1'], weights['Theta2']
print('theta1 :', theta1.shape)
print('theta2 :', theta2.shape)
params = np.r_[theta1.ravel(), theta2.ravel()]
print('params :', params.shape)

theta1 : (25, 401)
theta2 : (10, 26)
params : (10285,)


#### Neural Network
Input layer size = 400 (20x20 pixels) <br>
Hidden layer size = 25 <br>
Number of labels = 10

### Neural Networks - Feed Forward and Cost Function

In [7]:
def sigmoid(z):
    return(1 / (1 + np.exp(-z)))

#### Sigmoid gradient
#### $$ g'(z) = g(z)(1 - g(z))$$
where $$ g(z) = \frac{1}{1+e^{-z}}$$

In [8]:
def sigmoidGradient(z):
    return(sigmoid(z)*(1-sigmoid(z)))

In [9]:
[sigmoidGradient(z) for z in [-1, -0.5, 0, 0.5, 1]]

[0.19661193324148185,
 0.23500371220159449,
 0.25,
 0.23500371220159449,
 0.19661193324148185]

#### Cost Function 
#### $$ J(\theta) = \frac{1}{m}\sum_{i=1}^{m}\sum_{k=1}^{K}\big[-y^{(i)}_{k}\, log\,(( h_\theta\,(x^{(i)}))_k)-(1-y^{(i)}_k)\,log\,(1-h_\theta(x^{(i)}))_k)\big]$$

#### Regularized Cost Function
#### $$ J(\theta) = \frac{1}{m}\sum_{i=1}^{m}\sum_{k=1}^{K}\bigg[-y^{(i)}_{k}\, log\,(( h_\theta\,(x^{(i)}))_k)-(1-y^{(i)}_k)\,log\,(1-h_\theta(x^{(i)}))_k)\bigg] + \frac{\lambda}{2m}\bigg[\sum_{j=1}^{25}\sum_{k=1}^{400}(\Theta_{j,k}^{(1)})^2+\sum_{j=1}^{10}\sum_{k=1}^{25}(\Theta_{j,k}^{(2)})^2\bigg]$$

In [187]:
def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, features, classes, L):
    
    # When comparing to Octave code note that Python uses zero-indexed arrays.
    # But because Numpy indexing does not include the right side, the code is the same anyway.
    theta1 = nn_params[0:(hidden_layer_size*(input_layer_size+1))].reshape(hidden_layer_size,(input_layer_size+1))
    theta2 = nn_params[(hidden_layer_size*(input_layer_size+1)):].reshape(num_labels,(hidden_layer_size+1))

    m = features.shape[0]
    y_matrix = pd.get_dummies(classes.ravel()).as_matrix() 
    
    # Cost
    a1 = features # 5000x401
        
    z2 = theta1.dot(a1.T) # 25x401 * 401x5000 = 25x5000 
    a2 = np.c_[np.ones((features.shape[0],1)),sigmoid(z2.T)] # 5000x26 
    
    z3 = theta2.dot(a2.T) # 10x26 * 26x5000 = 10x5000 
    a3 = sigmoid(z3) # 10x5000
    
    J = -1*(1/m)*np.sum((np.log(a3.T)*(y_matrix)+np.log(1-a3).T*(1-y_matrix))) + \
        (L/(2*m))*(np.sum(np.square(theta1[:,1:])) + np.sum(np.square(theta2[:,1:])))

    # Gradients
    d3 = a3.T - y_matrix # 5000x10
    d2 = theta2[:,1:].T.dot(d3.T)*sigmoidGradient(z2) # 25x10 *10x5000 * 25x5000 = 25x5000
    
    delta1 = d2.dot(a1) # 25x5000 * 5000x401 = 25x401
    delta2 = d3.T.dot(a2) # 10x5000 *5000x26 = 10x26
    
    theta1_ = np.c_[np.ones((theta1.shape[0],1)),theta1[:,1:]]
    theta2_ = np.c_[np.ones((theta2.shape[0],1)),theta2[:,1:]]
    
    theta1_grad = delta1/m + (theta1_*L)/m
    theta2_grad = delta2/m + (theta2_*L)/m
    
    return(J, theta1_grad, theta2_grad)

In [184]:
# Regularization parameter = 0
nnCostFunction(params, 400, 25, 10, X, y, 0)[0]

[[-0.023 -0.     0.    ..., -0.    -0.     0.   ]
 [-0.098  0.    -0.    ..., -0.     0.     0.   ]
 [ 0.116 -0.     0.    ..., -0.    -0.    -0.   ]
 ..., 
 [-0.183 -0.    -0.    ...,  0.    -0.     0.   ]
 [-0.702  0.     0.    ..., -0.001  0.     0.   ]
 [-0.351  0.    -0.    ..., -0.    -0.     0.   ]]


0.28762916516131892

In [172]:
# Regularization parameter = 1
nnCostFunction(params, 400, 25, 10, X, y, 1)[0]

0.38376985909092365

In [190]:
nn = (np.arange(1,7)/10).reshape(2,3)
nn

array([[ 0.1,  0.2,  0.3],
       [ 0.4,  0.5,  0.6]])

In [134]:
XX = np.c_[np.array([[1],[1],[1]]), np.cos(np.arange(1,7).reshape(3,2))]
XX

array([[ 1.        ,  0.54030231, -0.41614684],
       [ 1.        , -0.9899925 , -0.65364362],
       [ 1.        ,  0.28366219,  0.96017029]])

In [135]:
yy = np.array([4,2,3]).reshape(-1,1)
yy

array([[4],
       [2],
       [3]])

In [186]:
nnCostFunction(nn, 2, 2, 4, XX , yy, 4)

[[ 0.1  0.2  0.3]
 [ 0.4  0.5  0.6]]


ValueError: operands could not be broadcast together with shapes (3,4) (3,3) 

In [164]:
np.c_[np.zeros((3,1)), pd.get_dummies(yy.ravel()).as_matrix()]

array([[ 0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.]])